In [16]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import mahotas

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

np.random.seed(37)

In [17]:
import itertools

indexes = list(itertools.product([0,1,2,3,4,5,6,7],[0,1,2],[0,1,2]))
print(indexes)

[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 2, 0), (0, 2, 1), (0, 2, 2), (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 2, 0), (1, 2, 1), (1, 2, 2), (2, 0, 0), (2, 0, 1), (2, 0, 2), (2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 2, 0), (2, 2, 1), (2, 2, 2), (3, 0, 0), (3, 0, 1), (3, 0, 2), (3, 1, 0), (3, 1, 1), (3, 1, 2), (3, 2, 0), (3, 2, 1), (3, 2, 2), (4, 0, 0), (4, 0, 1), (4, 0, 2), (4, 1, 0), (4, 1, 1), (4, 1, 2), (4, 2, 0), (4, 2, 1), (4, 2, 2), (5, 0, 0), (5, 0, 1), (5, 0, 2), (5, 1, 0), (5, 1, 1), (5, 1, 2), (5, 2, 0), (5, 2, 1), (5, 2, 2), (6, 0, 0), (6, 0, 1), (6, 0, 2), (6, 1, 0), (6, 1, 1), (6, 1, 2), (6, 2, 0), (6, 2, 1), (6, 2, 2), (7, 0, 0), (7, 0, 1), (7, 0, 2), (7, 1, 0), (7, 1, 1), (7, 1, 2), (7, 2, 0), (7, 2, 1), (7, 2, 2)]


In [18]:
def normalize(values, bounds):
    return [bounds['desired']['lower'] + (x - bounds['actual']['lower']) * (bounds['desired']['upper'] - bounds['desired']['lower']) / (bounds['actual']['upper'] - bounds['actual']['lower']) for x in values]

In [19]:
def get_F(h, s, v):
    h_norm = normalize(
        h,
        {'actual': {'lower': 0, 'upper': 179}, 'desired': {'lower': 0, 'upper': 360}}
    )
    s_norm = normalize(
        s,
        {'actual': {'lower': 0, 'upper': 255}, 'desired': {'lower': 0, 'upper': 1}}
    )
    v_norm = normalize(
        v,
        {'actual': {'lower': 0, 'upper': 255}, 'desired': {'lower': 0, 'upper': 1}}
    )
    
    h_norm = np.array(h_norm).flatten()
    s_norm = np.array(s_norm).flatten()
    v_norm = np.array(v_norm).flatten()
    
    h_d = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0}
    s_d = {0:0, 1:0, 2:0}
    v_d = {0:0, 1:0, 2:0}

    for i in range(len(h_norm)):
        H = h_norm[i]
        ha = None
        if H >= 316 or H <= 20:
            ha = 0
        elif H >= 21 and H <= 40:
            ha = 1
        elif H >= 41 and H <= 75:
            ha = 2
        elif H >= 76 and H <= 155:
            ha = 3
        elif H >= 156 and H <= 190:
            ha = 4
        elif H >= 191 and H <= 270:
            ha = 5
        elif H >= 271 and H <= 295:
            ha = 6
        elif H >= 296 and H <= 315:
            ha = 7
        if ha not in h_d:
            h_d[ha] = 1
        else:
            h_d[ha] += 1

    for i in range(len(s_norm)):
        S = s_norm[i]
        sa = None
        if S >= 0 and S <= 0.2:
            sa = 0
        elif S > 0.2 and S <= 0.7:
            sa = 1
        elif S > 0.7 and S <= 1:
            sa = 2
        if sa not in s_d:
            s_d[sa] = 1
        else:
            s_d[sa] += 1

    for i in range(len(v_norm)):
        V = v_norm[i]
        va = None
        if V >= 0 and V <= 0.2:
            va = 0
        elif V > 0.2 and V <= 0.7:
            va = 1
        elif V > 0.7 and V <= 1:
            va = 2
        if va not in v_d:
            v_d[va] = 1
        else:
            v_d[va] += 1
            
    F = np.zeros((72,))
    
    for i in range(8):
        for j in range(3):
            for k in range(3):
                F[indexes.index((i, j, k))] = 9*h_d[i] + 3*s_d[j] + v_d[k]
    return F

In [30]:
def save_video2(fname, frames, key_frames):
    count = 0

    cur_dir = fname[0:-4]
    print(cur_dir)
    os.mkdir(cur_dir)

    count = 0

    for i in range(len(frames)):
        if count in key_frames:
            if(cv2.imwrite(cur_dir + "\\" + str(count) + ".jpg", frames[i])):
                print(cur_dir + "\\" + str(count) + ".jpg")
        count += 1
    
    return

In [34]:
def save_video(fname, key_frames):
    count = 0

    cur_dir = fname[0:-4]
    print(cur_dir)
    os.mkdir(cur_dir)

    vidcap = cv2.VideoCapture(fname)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    success, img = vidcap.read()
    print('Read a new frame: ', success)

    while success:
        if count in key_frames:
            print(f'{count} in key frame')
            if(cv2.imwrite(cur_dir + "\\" + str(count) + ".jpg", img)):
                print(cur_dir + "\\" + str(count) + ".jpg")
            #plt.imshow(img)
            #plt.show()
        success, img = vidcap.read()
        #print('Read a new frame: ', success)
        count += 1
    vidcap.release()
    
    return

def presample_video(fname):
    vidcap = cv2.VideoCapture(fname)
    success, img = vidcap.read()
    fps = vidcap.get(cv2.CAP_PROP_FPS)

    #print(success)
    sampling_rate = 1

    count = 0
    frames = []
    frames.append(img)
    
    while success:
        if(count%(int(fps//sampling_rate)) == 0):
            frames.append(img)
            # print(count)
        count += 1
        
        success, img= vidcap.read()
    
    vidcap.release()
    
    return frames
        
def process_video(fname):
    frames = presample_video(fname)
    colour_features = []
    count = 0
    
    for frame in frames:
        img2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        img3 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        h, s, v = img2[:,:,0], img2[:,:,1], img2[:,:,2]
        F1 = get_F(h, s, v)
        F1_norm = normalize(
            F1,
            {'actual': {'lower': 0, 'upper': 1000000}, 'desired': {'lower': 0, 'upper': 1}}
        )
        thresh = cv2.threshold(img3, 75, 200, cv2.THRESH_BINARY)[1]
        F2 = np.append(mahotas.features.zernike_moments(thresh, 250), cv2.HuMoments(cv2.moments(img3)).flatten())
        #print(F)
        F = np.append(F1_norm, F2)
        colour_features.append(F)
        if(count%10 == 0):
            print("{count} images done".format(count=count))
        count += 1
        
    best_i = 0
    best_sc = -1

    for i in range(3,21):
        kmeans = KMeans(n_clusters=i, random_state=0).fit(colour_features)
        sc = silhouette_score(colour_features, kmeans.labels_)
        if(sc > best_sc):
            best_i = i
            best_sc = sc
        print(i, silhouette_score(colour_features, kmeans.labels_))

    kmeans = KMeans(n_clusters=best_i, random_state=0).fit(colour_features)
    cents = kmeans.cluster_centers_

    print(len(colour_features), len(kmeans.labels_))

    min_dict = {}
    min_frame= {}
    for i in range(len(kmeans.labels_)):
        t = np.linalg.norm(colour_features[i]-cents[kmeans.labels_[i]])
        if kmeans.labels_[i] not in min_dict:
            min_dict[kmeans.labels_[i]] = t
            min_frame[kmeans.labels_[i]] = i
        else:
            if(t < min_dict[kmeans.labels_[i]]):
                min_dict[kmeans.labels_[i]] = t
                min_frame[kmeans.labels_[i]] = i

    print(min_frame)

    key_frames = list(min_frame.values())
    
    save_video2(fname, frames, key_frames)


In [35]:
for root, dirs, files in os.walk("C:\\Users\\Souvik\\Desktop\\CHECK\\vid1", topdown=False):
    for name in files:
        print(os.path.join(root, name))
        process_video(os.path.join(root, name))

C:\Users\Souvik\Desktop\CHECK\vid1\ucf1.avi
0 images done
3 0.41688455385427947
4 0.4375848997521425
5 0.3667113240152011
6 0.30584839575535994
7 0.24999995454141624
8 0.24999995454141624


C:\Users\Souvik\AppData\Local\Temp\ipykernel_15320\1792267015.py:78: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=i, random_state=0).fit(colour_features)


ValueError: n_samples=8 should be >= n_clusters=9.